B1: find all the disjoint index groups where r2 is constant whenever j in I^m

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from PIL import Image
import os
import argparse
import time
from tqdm import trange
from tensorboardX import SummaryWriter
from collections import OrderedDict
from torchsummary import summary

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms

import medmnist
from medmnist import RetinaMNIST
from medmnist import INFO, Evaluator

In [40]:
info = INFO["retinamnist"]
DataClass = getattr(medmnist, info['python_class'])
data_transform = transforms.Compose(
        [transforms.ToTensor(),
        transforms.Normalize(mean=[.5], std=[.5])])
train_dataset = DataClass(split='train', transform=data_transform, download=True, as_rgb=True)
train_dataset = train_dataset #.cpu().detach().numpy()
X_train = []
y_train = []
for i in range(len(train_dataset)):
    X_train.append(train_dataset[i][0].cpu().detach().numpy())
    y_train.append(train_dataset[i][1])
print(type(train_dataset))
print(X_train)
X_train = np.array(X_train)
y_train = np.array(y_train)

Using downloaded and verified file: /home/matthijs/.medmnist/retinamnist.npz
<class 'medmnist.dataset.RetinaMNIST'>
[array([[[-0.9843137 , -0.96862745, -0.96862745, ..., -1.        ,
         -1.        , -1.        ],
        [-0.94509804, -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-0.9529412 , -1.        , -1.        , ..., -0.9843137 ,
         -0.99215686, -0.99215686],
        ...,
        [-1.        , -0.99215686, -0.9764706 , ..., -0.99215686,
         -0.99215686, -0.99215686],
        [-0.96862745, -0.99215686, -0.9764706 , ..., -0.99215686,
         -0.99215686, -1.        ],
        [-0.92156863, -0.9843137 , -0.9764706 , ..., -0.99215686,
         -1.        , -1.        ]],

       [[-1.        , -0.9764706 , -0.9764706 , ..., -0.9764706 ,
         -0.9764706 , -0.9764706 ],
        [-0.96862745, -1.        , -1.        , ..., -0.9843137 ,
         -0.99215686, -0.99215686],
        [-0.9607843 , -1.        , -1.        , ...,

In [62]:
from tensorflow.keras.models import Sequential
class CustomModel(Sequential):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)
        print(gradients)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}


In [63]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
X_train_flatten = X_train.reshape((X_train.shape[0], X_train.shape[1]*X_train.shape[2]*X_train.shape[3]))
L1 = 300
L2 = 200
L3 = 120
L4 = 50
model = CustomModel()
model.add(Dense(L1, input_shape=(X_train_flatten.shape[1:])))
model.add(Dense(L2, activation='relu'))
model.add(Dense(L3))
model.add(Dense(L4, activation='relu'))
model.add(Dense(5, activation='softmax'))
model.compile(optimizer="Adam", loss=tf.keras.metrics.sparse_categorical_crossentropy, metrics=["acc"])


In [64]:


my_callback = myCallback()
model.fit(X_train_flatten, y_train, epochs=1)

[<tf.Tensor 'gradient_tape/custom_model_1/dense_40/MatMul/MatMul:0' shape=(2352, 300) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_40/BiasAdd/BiasAddGrad:0' shape=(300,) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_41/MatMul/MatMul_1:0' shape=(300, 200) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_41/BiasAdd/BiasAddGrad:0' shape=(200,) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_42/MatMul/MatMul_1:0' shape=(200, 120) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_42/BiasAdd/BiasAddGrad:0' shape=(120,) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_43/MatMul/MatMul_1:0' shape=(120, 50) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_43/BiasAdd/BiasAddGrad:0' shape=(50,) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_44/MatMul/MatMul_1:0' shape=(50, 5) dtype=float32>, <tf.Tensor 'gradient_tape/custom_model_1/dense_44/BiasAdd/BiasAddGrad:0' shape=(5,) dty